In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from pathlib import Path
import scipy.constants as sc
from math import pi
import scipy.integrate as integrate
import sympy as sp
from matplotlib.font_manager import FontProperties
import uproot
import matplotlib.cm as cm
import matplotlib.ticker as ticker

In [2]:
#Plotting functions
def OnePlot(xlabel,ylabel,xscale,yscale,nxaxis,xlabel_secondary,xscale_secondary,Savename):
    ax1.grid(True,alpha=0.7,ls='dotted')
    ax1.legend(fontsize = 9)
    ax1.set_xlabel(xlabel, fontsize=9)
    ax1.set_ylabel(ylabel,fontsize=9)
    if xscale == "log":
        ax1.set_xscale("log")
    if yscale == "log":
        ax1.set_yscale("log")
    if nxaxis == 2:
        ax1_secondary = ax1.twiny()
        ax1_secondary.set_xlabel(xlabel_secondary, fontsize=9)
        xlim1_secondary = secondary_xaxis_transform(np.array(ax1.get_xlim()))
        ax1_secondary.set_xlim(xlim1_secondary)
    if xscale_secondary == "log":
        ax1_secondary.set_xscale("log")
    plt.rcParams.update({'font.size': 7, 'font.family' : 'serif'})
    plt.tight_layout()
    fig.savefig("C:/Users/maria/Documents/Uni/Bachelorarbeit/Coden/Leibelt_Bilder/" + Savename + ".pdf", bbox_inches = "tight")
    fig.savefig("C:/Users/maria/Documents/Uni/Bachelorarbeit/Coden/Leibelt_Bilder/" + Savename + ".png", dpi=700, bbox_inches = "tight")
    plt.show()
    
def TwoPlots(xlabel1,ylabel1,xscale1,yscale1,nxaxis1,xlabel_secondary1,xscale1_secondary,xlabel2,ylabel2,xscale2,yscale2,nxaxis2,xlabel_secondary2,xscale2_secondary,Savename):
    ax1.grid(True,alpha=0.7,ls='dotted')
    # ax1.legend(fontsize = 9)
    ax1.set_xlabel(xlabel1, fontsize=9)
    ax1.set_ylabel(ylabel1,fontsize=9)
    ax2.grid(True,alpha=0.7,ls='dotted')
    # ax2.legend(fontsize = 9)
    ax2.set_xlabel(xlabel2, fontsize=9)
    ax2.set_ylabel(ylabel2,fontsize=9)
    if xscale1 == "log":
        ax1.set_xscale("log")
    if yscale1 == "log":
        ax1.set_yscale("log")
    if xscale2 == "log":
        ax2.set_xscale("log")
    if yscale2 == "log":
        ax2.set_yscale("log")
    if nxaxis1 == 2:
        ax1_secondary = ax1.twiny()
        ax1_secondary.set_xlabel(xlabel_secondary1, fontsize=9)
        xlim1_secondary = secondary_xaxis_transform(np.array(ax1.get_xlim()))
        ax1_secondary.set_xlim(xlim1_secondary)
        
    if nxaxis2 == 2:
        ax2_secondary = ax2.twiny()
        ax2_secondary.set_xlabel(xlabel_secondary2, fontsize=9)
        xlim2_secondary = secondary_xaxis_transform(np.array(ax2.get_xlim()))
        ax2_secondary.set_xlim(xlim2_secondary)

    if xscale1_secondary == "log":
        ax1_secondary.set_xscale("log")
    if xscale2_secondary == "log":
        ax2_secondary.set_xscale("log")

    plt.rcParams.update({'font.size': 7, 'font.family' : 'serif'})
    plt.tight_layout()
    fig.savefig("C:/Users/maria/Documents/Uni/Bachelorarbeit/Coden/Leibelt_Bilder/" + Savename + ".pdf", bbox_inches = "tight")
    fig.savefig("C:/Users/maria/Documents/Uni/Bachelorarbeit/Coden/Leibelt_Bilder/" + Savename + ".png", dpi=700, bbox_inches = "tight")
    plt.show()

def secondary_xaxis_transform(x):
    return x * 3.3 

In [3]:
#Define used functions

#photon arrival time spectra
def l(t,A,kx,kxM,Type,Ntot,kI):
    NS = constants["sfrac"]*Ntot
    NT = (1-constants["sfrac"])*Ntot
    CS= A/constants["tauS"] + kx
    CT = A/constants["tauT"] + kx
    lambdaM = 1/constants["tauM"] + kxM + constants["kqM"] + kI
    lambdaS = 1/constants["tauS"] + kx + constants["kqS"] + kI
    lambdaT = 1/constants["tauT"] + kx + constants["kqT"] + kI
    lambdaX = 1/constants["tauX"] + constants["kqX"] + kI

    
    l1 = (NS * (1-A) / constants["tauS"]) * np.exp(-t*lambdaS)
    
    l2 = (NT * (1-A) / constants["tauT"]) * np.exp(-t*lambdaT)
    
    M1 = NS*CS/(lambdaS-lambdaM) * (np.exp(-t*lambdaM)-np.exp(-t*lambdaS))
    M3 = NT*CT/(lambdaT-lambdaM) * (np.exp(-t*lambdaM)-np.exp(-t*lambdaT))
    M = M1 + M3
    l3 = M/constants["tauM"]
    
    X1 = NS*CS*kx/(lambdaS-lambdaM) * ( (np.exp(-t*lambdaM)-np.exp(-t*lambdaX))/(lambdaX-lambdaM) - (np.exp(-t*lambdaS)-np.exp(-t*lambdaX))/(lambdaX-lambdaS) )
    X3 = NT*CT*kx/(lambdaT-lambdaM) * ( (np.exp(-t*lambdaM)-np.exp(-t*lambdaX))/(lambdaX-lambdaM) - (np.exp(-t*lambdaT)-np.exp(-t*lambdaX))/(lambdaX-lambdaT) )
    X = X1 + X3
    l4 = X/constants["tauX"]
    
    if Type == "S":
        l = l1  
    if Type == "T":
        l = l2    
    if Type == "M":
        l = l3 
    if  Type == "X":
        l = l4
    if Type == "All":
        l = l1 + l2 + l3 + l4
    
    return l


#Light yield: photon arrival time spectra integrated from 0-infinity
def Lxe(xecon,Type,Distance,Ntot,kI,kx):
    kxM = kx

    lambda1fit = 12.69 * 0.1/xecon
    lambda2fit = 741.4 * 0.1/xecon
    T = constants["Aconst"]*np.exp(-Distance/lambda1fit)+(1-constants["Aconst"])*np.exp(-Distance/lambda2fit)
    A = 1-T

    NS = constants["sfrac"]*Ntot
    NT = (1-constants["sfrac"])*Ntot
    CS= A/constants["tauS"] + kx
    CT = A/constants["tauT"] + kx

    lambdaM = 1/constants["tauM"] + kxM + constants["kqM"]+ kI
    lambdaS = 1/constants["tauS"] + kx + constants["kqS"] + kI
    lambdaT = 1/constants["tauT"] + kx + constants["kqT"] + kI
    lambdaX = 1/constants["tauX"] + constants["kqX"] + kI
    
    L1 = (NS * (1-A) / constants["tauS"]) * (1/lambdaS) 
    L2 = (NT*(1-A)/constants["tauT"])* (1/lambdaT)
    L3 = 1/constants["tauM"]*(NS*CS/(lambdaS-lambdaM)*(1/lambdaM-1/lambdaS)+NT*CT/(lambdaT-lambdaM)*(1/lambdaM-1/lambdaT))
    L4 = 1/constants["tauX"]*( (NS*CS*kx)/(lambdaS-lambdaM)*(1/(lambdaX-lambdaM)*(1/lambdaM-1/lambdaX)-1/(lambdaX-lambdaS)*(1/lambdaS-1/lambdaX))
                  +(NT*CT*kx)/(lambdaT-lambdaM)*(1/(lambdaX-lambdaM)*(1/lambdaM-1/lambdaX)-1/(lambdaX-lambdaT)*(1/lambdaT-1/lambdaX)) )
    
    if Type == "S":
        Lxe = L1  
    if Type == "T":
        Lxe = L2    
    if Type == "M":
        Lxe = L3 
    if  Type == "X":
        Lxe = L4
    if Type == "All":
        Lxe = L1+L2+L3+L4
        
    return Lxe

#Light yield dependent on xenon concentration and xenon quenching strength kx: photon arrival time spectra integrated from 0-infinity
def Lxekx(xecon,Type,Distance,Ntot,kI,kx):
    kxM = kx
    if xecon == 0:
        A=0
    else:
        lambda1fit = 12.69 * 0.1/xecon
        lambda2fit = 741.4 * 0.1/xecon
        T = constants["Aconst"]*np.exp(-Distance/lambda1fit)+(1-constants["Aconst"])*np.exp(-Distance/lambda2fit)
        A = 1-T

    NS = constants["sfrac"]*Ntot
    NT = (1-constants["sfrac"])*Ntot
    CS= A/constants["tauS"] + kx
    CT = A/constants["tauT"] + kx

    lambdaM = 1/constants["tauM"] + kxM + constants["kqM"]+ kI
    lambdaS = 1/constants["tauS"] + kx + constants["kqS"] + kI
    lambdaT = 1/constants["tauT"] + kx + constants["kqT"] + kI
    lambdaX = 1/constants["tauX"] + constants["kqX"] + kI
    
    L1 = (NS * (1-A) / constants["tauS"]) * (1/lambdaS) 
    L2 = (NT*(1-A)/constants["tauT"])* (1/lambdaT)
    L3 = 1/constants["tauM"]*(NS*CS/(lambdaS-lambdaM)*(1/lambdaM-1/lambdaS)+NT*CT/(lambdaT-lambdaM)*(1/lambdaM-1/lambdaT))
    L4 = 1/constants["tauX"]*( (NS*CS*kx)/(lambdaS-lambdaM)*(1/(lambdaX-lambdaM)*(1/lambdaM-1/lambdaX)-1/(lambdaX-lambdaS)*(1/lambdaS-1/lambdaX))
                  +(NT*CT*kx)/(lambdaT-lambdaM)*(1/(lambdaX-lambdaM)*(1/lambdaM-1/lambdaX)-1/(lambdaX-lambdaT)*(1/lambdaT-1/lambdaX)) )
    
    if Type == "S":
        Lxekx = L1  
    if Type == "T":
        Lxekx = L2    
    if Type == "M":
        Lxekx = L3 
    if  Type == "X":
        Lxekx = L4
    if Type == "All":
        Lxekx = L1+L2+L3+L4
        
    return Lxekx

#Functions needed to perform fits to data

#photon arrival time spectra with padding for t<0 (account for the finite time resolution of the detector)
def lpad(t,A,kx,kxM,Ntot,kI):
    lpad = np.where(t < 0, 0, l(t,A,kx,kxM,'All',Ntot,kI))
    return lpad

#how the fit would look like without the convolution with a gaussian
def fitohnegauss(t,A,kx,kxM,Ntot,kI,c):
    fitohnegauss = np.where(t < 0, 0, l(t,A,kx,kxM,'All',Ntot,kI))+c
    return fitohnegauss

#Convolution----------------------------------------------------------------------------------------
def gaussian(x, mu, sigma):
    gauss = (1/(sigma * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x - mu) / sigma)**2)
    gaussnorm = gauss / np.sum(gauss)
    return gaussnorm

def ConvFit(x,xgauss,mu,sigma,A,kx,kxM,Ntot,kI,c):
    model=lpad(x, A, kx, kxM, Ntot, kI) + c
    gauss=gaussian(xgauss, mu, sigma)
    conv = np.convolve(gauss,model, mode='valid')
    return conv
#----------------------------------------------------------------------------------------------------

#check if it is an even number
def ist_gerade(zahl):
    return zahl % 2 == 0


In [4]:
xecon_mass = np.array([0,3,10,50,100,300]) #xenon concentration in ppm(mass)
xecon_mol = np.array([0,3,10,50,100,300]) / 3.3 #xenon concentration in ppm(mol)
xeconcont = np.arange(0.0,300.01,0.01)    #xenon concentration in ppm(mass) continous
xeconpey = np.arange(0.01,300.01,0.01)    #xenon concentration in ppm(mass) continous for photoelectron yield
xeconcont_mol = np.arange(0.0,300.01,0.01) / 3.3    #xenon concentration in ppm(mol) continous
xeconly_mol = np.arange(0.01,300.01,0.01) / 3.3    #xenon concentration in ppm(mol) continous for rel. light yield yield
xeconpey_mol = np.arange(0.01,300.01,0.01) / 3.3    #xenon concentration in ppm(mol) continous for photoelectron yield
xecon_mass_log = np.array([0.01,3,10,50,100,300])
xecon_mol_log = np.array([0.01,3,10,50,100,300]) / 3.3

Data = np.array([33.63,31.10,39.15,54.51,59.33,64.99]) #see Master's Thesis of C. Vogl; DOI: 10.13140/RG.2.2.23118.54084
Data_error = np.array([0.05, 0.03, 0.03,0.05,0.06,0.04])
Data_norm = Data/Data[5]
Data_norm_error = Data_error/Data[5]
path = ['000ppm','003ppm','010ppm','050ppm','100ppm','300ppm']

#Colorblind-friendly palette
darkblue = '#4477AA'
purple = '#AA3377'
cyan = '#66CCEE'
green = '#228833'
yellow = '#CCBB44'
grey =  '#BBBBBB'
pink = '#EE6677'
Color = ['k',darkblue,cyan,green,yellow,pink,purple,grey] #colorblind-friendly palette
Colortype = np.array([pink,darkblue,green,yellow,'k'])

#Labels
Conname = np.array([r"0.00 $\mathrm{\mu mol/mol}$",r"0.91 $\mathrm{\mu mol/mol}$",r"3.03 $\mathrm{\mu mol/mol}$",r"15.15 $\mathrm{\mu mol/mol}$",r"30.30 $\mathrm{\mu mol/mol}$",r"90.91 $\mathrm{\mu mol/mol}$"])
Xe_mol_label = r"Xe concentration [$\mathrm{\mu mol/mol}$]"
Xe_g_label = r"Xe concentration [$\mathrm{\mu g/g}$]"
pe_label = "rel. photoelectron yield [pe]"
tau3_label = "effective triplet lifetime [ns]"
time_label = "time [ns]"
relly_label = "rel. light yield"
countlabel = "counts (a.u.)"
Type =['S','T','X','M','All']
Typename = ['Singlets','Triplets','Xenon','Mixed states','All contributions']

x = [0.93,15.,40.,75.] #distances

In [5]:
constants = {
    "sfrac": 0.14, #singlet fraction
    "PDE128": 0.143, #PDE CORRECTION for a 128nm photon
    "PDE150": 0.235, #PDE CORRECTION for a 150nm photon
    "PDE175": 0.238, #PDE CORRECTION for a 175nm photon
    "tauS": 7, #argon excimer singlet lifetime
    "tauT": 1600, #argon excimer triplet lifetime
    "tauM": 4700, #mixed state (ArXe)* lifetime
    "tauX": 20, #xenon excimer lifetime
    "kq": 1.3 * 1e-4, #self-quenching rate
    "kqS": 1.3 * 1e-4, 
    "kqT": 1.3 * 1e-4,
    "kqM": 1.3 * 1e-4,
    "kqX": 1.3 * 1e-4,
    "Aconst": 0.62, 
    "kI_average": 3.08 * 1e-4, #average of the calculated bounds for the impurity quenching rate
    "kI_lower_bound": 2.93 * 1e-4, 
    "kI_upper_bound": 3.73 * 1e-4
}

In [ ]:
#Photon arrival time spectra - t: 0-10000 - Distance
print('--------------------------------------------------------')
print('Photon arrival time spectra - t: 0-10000 - Distance')
print('--------------------------------------------------------')
tmax = 10000
t = np.array(range(0, tmax))
kx = 2.9 * 1e-4 * xecon_mol #xenon quenching rate
kxM = kx 
Ntot = 1

for i in range(len(x)):
    Distance = x[i]
    print(Distance)
    
    strDistance = str(x[i])
    fig, (ax1,ax2) = plt.subplots(1,2,figsize=(7, 3))
    for j in range(0,len(xecon_mol)):
        if j == 0:
            A = 0 #Absorption of scintillation light
        else:
            lambda1fit = 12.69 * 0.1/xecon_mol[j]
            lambda2fit = 741.4 * 0.1/xecon_mol[j]
            T = constants["Aconst"]*np.exp(-Distance/lambda1fit)+(1-constants["Aconst"])*np.exp(-Distance/lambda2fit)
            A = 1-T #Absorption of scintillation light
        y = l(t,A,kx[j],kxM[j],'All',Ntot,constants["kI_average"])
        ax1.plot(t, y, Color[j+1], label = Conname[j])
        ax2.plot(t, y, Color[j+1], label = Conname[j])
    ax1.set_ylim(1e-5,1e-1)
    #plt.xlim(-50,3000)
    ax2.set_ylim(1e-5,1e-1)
    ax2.set_xlim(-50,9e2)
    legend1 = ax1.legend(fontsize=9)
    legend1.set_title(strDistance + " cm", prop = FontProperties(size=10))
    TwoPlots(time_label,countlabel,"no","log",1,"no","no",time_label,countlabel,"no","log",1,"no","no","Leibelt_Photonarr_zoom"+ strDistance)

In [ ]:
#Photon Arrival Time spectra showing the contributions 
tmax = 10000
t = np.array(range(0, tmax))

Distance = 15.
Ntot = 1
xecon_contributions = np.array([0,3,50])/3.3

for i in range(1,len(xecon_contributions)):
    if i == len(xecon_contributions)-1:
        break
    if i == 0:
        A1 = 0
        kx1 = 0
        kxM1 = 0
        lambda1fit2 = 12.69 * 0.1/xecon_contributions[i+1]
        lambda2fit2 = 741.4 * 0.1/xecon_contributions[i+1]
        kx2 = 2.9 * 1e-4 * xecon_contributions[i+1]
        kxM2 = kx2
        T2 = constants["Aconst"]*np.exp(-Distance/lambda1fit2)+(1-constants["Aconst"])*np.exp(-Distance/lambda2fit2)
        A2 = 1-T2
    else:
        lambda1fit1 = 12.69 * 0.1/xecon_contributions[i]
        lambda2fit1 = 741.4 * 0.1/xecon_contributions[i]
        kx1 = 2.9 * 1e-4 * xecon_contributions[i]
        kxM1 = kx1
        T1 = constants["Aconst"]*np.exp(-Distance/lambda1fit1)+(1-constants["Aconst"])*np.exp(-Distance/lambda2fit1)
        A1 = 1-T1
        lambda1fit2 = 12.69 * 0.1/xecon_contributions[i+1]
        lambda2fit2 = 741.4 * 0.1/xecon_contributions[i+1]
        kx2 = 2.9 * 1e-4 * xecon_contributions[i+1]
        kxM2 = kx2
        T2 = constants["Aconst"]*np.exp(-Distance/lambda1fit2)+(1-constants["Aconst"])*np.exp(-Distance/lambda2fit2)
        A2 = 1-T2

    fig, (ax1,ax2) = plt.subplots(1,2,figsize=(7, 3))
    for  j in range(0,len(Type)):
        if j == 4:
            y1 = l(t,A1,kx1,kxM1,Type[j],Ntot,constants["kI_average"])
            y2 = l(t,A2,kx2,kxM2,Type[j],Ntot,constants["kI_average"])
            ax1.plot(t, y1,"k--", label = Typename[j])
            ax2.plot(t, y2,"k--", label = Typename[j])
        else:
            y1 = l(t,A1,kx1,kxM1,Type[j],Ntot,constants["kI_average"])
            y2 = l(t,A2,kx2,kxM2,Type[j],Ntot,constants["kI_average"])
            ax1.plot(t, y1, Colortype[j], label = Typename[j])
            ax2.plot(t, y2, Colortype[j], label = Typename[j])
    ax1.set_ylim(1e-6,3e-2)
    ax2.set_ylim(1e-6,3e-2)
    legend1 = ax1.legend(fontsize=9)
    legend1.set_title(Conname[1], prop = FontProperties(size=10))
    legend2 = ax2.legend(fontsize = 9)
    legend2.set_title(Conname[3], prop = FontProperties(size=10))
    TwoPlots(time_label,countlabel,"no","log",1,"no","no",time_label,countlabel,"no","log",1,"no","no","Leibelt_Photonarr_contributions")

In [ ]:
# Relative light yield - t: 0-unendlich
Distance = 0.93
Ntot=1
kx = 2.9 * 1e-4 * xeconly_mol
fig, ax1 = plt.subplots(figsize=(4, 3))
L = Lxe(xeconly_mol,"All",Distance,Ntot,constants["kI_average"],kx)
norm_val = L[29999]
L_norm = norm_val * L
ax1.plot(xeconpey_mol, L_norm,color=Color[0], label = "New model")
#ax1.set_ylim(0.8,1.55)
OnePlot(Xe_mol_label,relly_label,"no","no",2,Xe_g_label,"no",'Leibelt_relly')


In [ ]:
#Relative Light Yield showing the contributions
Distance = 0.93
fig, (ax1,ax2) = plt.subplots(1, 2, figsize=(7, 3))
kx = 2.9 * 1e-4 * xeconly_mol
LAll = Lxe(xeconly_mol,"All",Distance,Ntot,constants["kI_average"],kx)
norm_val = 1/LAll[29999]
for i in range(0,len(Type)):
    y = Lxe(xeconly_mol,Type[i],Distance,Ntot,constants["kI_average"],kx) * norm_val
    ax1.plot(xeconly_mol, y, Colortype[i], label = Typename[i])
    ax2.plot(xeconly_mol, y, Colortype[i], label = Typename[i])
#OnePlot(Xe_mol_label,relly_label,"log","no",2,Xe_g_label,"log","Fields_relly_Types")
legend1 = ax1.legend(fontsize=9)
TwoPlots(Xe_mol_label,relly_label,"no","no",2,Xe_g_label,"no",Xe_mol_label,relly_label,"log","no",2,Xe_g_label,"log","Leibelt_relly_Types_norm")

In [ ]:
#Photoelectron Yield (pey)
Distance = 0.93
Ntot = 1
kx = 2.9 * 1e-4 * xeconpey_mol

for j in range(0,len(Type)-1):
    if Type[j] == 'S':
        yAllS = Lxe(xeconpey_mol,Type[j],Distance,Ntot,constants["kI_average"],kx)*constants["PDE128"]
    elif Type[j] == 'T':
        yAllT = Lxe(xeconpey_mol,Type[j],Distance,Ntot,constants["kI_average"],kx)*constants["PDE128"]
    elif Type[j] == 'M':
        yAllM = Lxe(xeconpey_mol,Type[j],Distance,Ntot,constants["kI_average"],kx)*constants["PDE150"]
    elif Type[j] == 'X':
        yAllX = Lxe(xeconpey_mol,Type[j],Distance,Ntot,constants["kI_average"],kx)*constants["PDE175"]
    else:
        PDE = -2000

yAll = yAllS + yAllT + yAllM + yAllX
norm_val = 1/yAll[29999]

#pey
fig, (ax1,ax2) = plt.subplots(1, 2, figsize=(7, 3))
ax1.plot(xeconpey_mol, yAllS*norm_val, Colortype[0], label = Typename[0])
ax1.plot(xeconpey_mol, yAllT*norm_val, Colortype[1], label = Typename[1])
ax1.plot(xeconpey_mol, yAllX*norm_val, Colortype[2], label = Typename[2])
ax1.plot(xeconpey_mol, yAllM*norm_val, Colortype[3], label = Typename[3])
ax1.plot(xeconpey_mol, yAll*norm_val, Colortype[4], label = Typename[4])

ax2.plot(xeconpey_mol, yAllS*norm_val, Colortype[0], label = Typename[0])
ax2.plot(xeconpey_mol, yAllT*norm_val, Colortype[1], label = Typename[1])
ax2.plot(xeconpey_mol, yAllX*norm_val, Colortype[2], label = Typename[2])
ax2.plot(xeconpey_mol, yAllM*norm_val, Colortype[3], label = Typename[3])
ax2.plot(xeconpey_mol, yAll*norm_val, Colortype[4], label = Typename[4])

legend1 = ax1.legend(fontsize=9)
TwoPlots(Xe_mol_label,pe_label,"no","no",2,Xe_g_label,"no",Xe_mol_label,pe_label,"log","no",2,Xe_g_label,"log","Leibelt_pey")

#pey without data
fig, (ax1,ax2) = plt.subplots(1, 2, figsize=(7, 3))
ax1.plot(xeconpey_mol, yAll*norm_val, 'k', label = 'New model')
ax2.plot(xeconpey_mol, yAll*norm_val, 'k', label = 'New model')
legend1 = ax1.legend(fontsize=9)
TwoPlots(Xe_mol_label,pe_label,"no","no",2,Xe_g_label,"no",Xe_mol_label,pe_label,"log","no",2,Xe_g_label,"log","Leibelt_pey_withoutData")

#pey compared to data
fig, (ax1,ax2) = plt.subplots(1, 2, figsize=(7, 3))
ax1.plot(xeconpey_mol, yAll*norm_val, 'k', label = 'New model')
ax1.errorbar(xecon_mol, Data_norm, Data_norm_error,fmt = '.',color = Color[6], label = 'LLAMA data')
ax2.plot(xeconpey_mol, yAll*norm_val, 'k', label = 'New model')
ax2.errorbar(xecon_mol_log, Data_norm, Data_norm_error,fmt = '.',color = Color[6], label = 'LLAMA data')
legend1 = ax1.legend(fontsize=9)
TwoPlots(Xe_mol_label,pe_label,"no","no",2,Xe_g_label,"no",Xe_mol_label,pe_label,"log","no",2,Xe_g_label,"log","Leibelt_pey_Data")


In [ ]:
#Fit to the data with fit parameters: kI - sigma,Ntot,c - with weighted bins and rebinned
Distance = 15.

kIarr= []
kIcalcarr = []
kxtheoarr = []

for i in range(0,len(path)):
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    print('i:',i,',',path[i])
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')

    #set parameters
    if i == 0:
        kx, kxM, A = 0, 0, 0
    else:
        kx = 2.9 * 1e-4 * xecon_mol[i]
        kxM = 2.9 * 1e-4 * xecon_mol[i]
        lambda1fit = 12.69 * 0.1/xecon_mol[i]
        lambda2fit = 741.4 * 0.1/xecon_mol[i]
        T = constants["Aconst"]*np.exp(-Distance/lambda1fit)+(1-constants["Aconst"])*np.exp(-Distance/lambda2fit)
        A = 1-T

    # Open the ROOT file and extract the histogram
    file = uproot.open("C:/Users/maria/Documents/Uni/Bachelorarbeit/Coden/out_stable/stable_" + path[i]+"/out.root")
    hist = file["hf2_4_1"]

    #  Get the bin contents and edges
    counts, bin_edges = hist.to_numpy()

    # Rebin
    rebin_factor = 2
    new_counts = np.add.reduceat(counts, np.arange(0, len(counts), rebin_factor))
    new_bin_edges = bin_edges[::rebin_factor]
    bin_centers = (new_bin_edges[:-1] + new_bin_edges[1:]) / 2

    # Generate bin centers with the specified step size
    step_size = new_bin_edges[1]-new_bin_edges[0]
    #print('stepsize: ',step_size)

    # Gauss x values
    xgauss = np.arange(-100, 110, step_size)
    #print('xgauss: ',xgauss)

    #x-values for the Fit and the data
    x_min, x_max = -200, 1e4
    x_pad_start, x_conv_max = -200,1e4
    #mu, sigma, Ntot, kI, c
    bounds = (
        [-100, 10, -np.inf, 0., 0],  # Lower bounds
        [100, 100, np.inf, 1e-2, np.inf]  # Upper bounds
    )
    boundscalc = (
        [-100, 10, -np.inf, constants["kI_lower_bound"], 0],  # Lower bounds
        [100, 100, np.inf, constants["kI_upper_bound"], np.inf]  # Upper bounds
    )
    if i == 0:
        xfit_min, xfit_max = -250, 10000
        initial_guesses = [1, 50, 4e6, 3e-4, 15]
    elif i == 1:
        xfit_min, xfit_max = -250, 10000
        initial_guesses = [1, 50, 4e6, 3e-4, 10]
    elif i == 2:
        xfit_min, xfit_max = -250, 5200
        initial_guesses = [1, 50, 4e6, 3e-4, 25]
    elif i == 3:
        xfit_min, xfit_max = -250, 2100
        initial_guesses = [1, 50, 4e6, 3e-4, 25]
        bounds = (
            [-100, 30, -np.inf, 0., 0],  # Lower bounds
            [100, 50, np.inf, 1e-2, np.inf]  # Upper bounds
        )
        boundscalc = (
            [-100, 30, -np.inf, constants["kI_lower_bound"], 0],  # Lower bounds
            [100, 50, np.inf, constants["kI_upper_bound"], np.inf]  # Upper bounds
        )
    elif i == 4:
        xfit_min, xfit_max = -250, 2250
        initial_guesses = [1, 50, 4e6, 3e-4, 20]
        bounds = (
            [-np.inf, 30, -np.inf, 0., 0],  # Lower bounds
            [np.inf, 50, np.inf, 1e-2, np.inf]  # Upper bounds
        )
        boundscalc = (
            [-np.inf, 30, -np.inf, constants["kI_lower_bound"], 0],  # Lower bounds
            [np.inf, 50, np.inf, constants["kI_upper_bound"], np.inf]  # Upper bounds
        )
    elif i == 5:
        xfit_min, xfit_max = -250, 1500
        initial_guesses = [1, 50, 4e6, 3e-4, 70]
        bounds = (
            [-100, -np.inf, -np.inf, 0., 0],  # Lower bounds
            [100, np.inf, np.inf, 1e-2, np.inf]  # Upper bounds
        )
        boundscalc = (
            [-100, -np.inf, -np.inf,constants["kI_lower_bound"], 0],  # Lower bounds
            [100, np.inf, np.inf, constants["kI_upper_bound"], np.inf]  # Upper bounds
        )

    # Apply the mask to filter data for x_min to x_max
    mask = (bin_centers >= x_min) & (bin_centers <= x_max)
    xdata = bin_centers[mask]
    ydata = new_counts[mask]

    mask_conv = (bin_centers >= x_pad_start) & (bin_centers <= x_conv_max)
    xdata_conv = bin_centers[mask_conv]
    ydata_conv = new_counts[mask_conv]

    mask_fit = (bin_centers >= xfit_min) & (bin_centers <= xfit_max)
    xdata_fit = bin_centers[mask_fit]
    ydata_fit = new_counts[mask_fit]
      
    ydata_fit_chopped = ydata_fit[3:][:-2]

    # Weight: sqrt(#binentries)
    weight = np.where(ydata_fit_chopped == 0, np.inf, np.sqrt(ydata_fit_chopped))  

    # Curve fitting
    popt_weighted, pcov_weighted = curve_fit(
        lambda t_, mu_fit, sigma_fit, Ntot_fit, kI_fit, c_fit: ConvFit(
            t_, xgauss, mu_fit, sigma_fit, A, kx, kxM, Ntot_fit, kI_fit, c_fit),
        xdata_fit, ydata_fit_chopped, p0=initial_guesses, bounds=bounds, sigma=weight, absolute_sigma=True
    )
    poptcalc_weighted, pcovcalc_weighted = curve_fit(
        lambda t_, mucalc_fit, sigmacalc_fit, Ntotcalc_fit, kIcalc_fit, ccalc_fit: ConvFit(
            t_, xgauss, mucalc_fit, sigmacalc_fit, A, kx, kxM, Ntotcalc_fit, kIcalc_fit, ccalc_fit),
        xdata_fit, ydata_fit_chopped, p0=initial_guesses, bounds=boundscalc, sigma=weight, absolute_sigma=True
    )

    mu_fit, sigma_fit, Ntot_fit, kI_fit, c_fit = popt_weighted
    mucalc_fit, sigmacalc_fit, Ntotcalc_fit, kIcalc_fit, ccalc_fit = poptcalc_weighted

    param_errors = np.sqrt(np.diag(pcov_weighted))
    param_errors_calc = np.sqrt(np.diag(pcovcalc_weighted))
    
    #print('-------------------------------------------------------------------------------------------')
    print('Guesses: ','mu: ',initial_guesses[0],'sigma: ',initial_guesses[1],'k_I: ',initial_guesses[3],'c: ',initial_guesses[4])
    print('mu_fit:    ',mu_fit,'        mucalc_fit:    ',mucalc_fit)
    print('sigma_fit: ',sigma_fit,'          sigmacalc_fit:  ',sigmacalc_fit)
    print('Ntot_fit:  ',Ntot_fit,'        Ntotcalc_fit:   ',Ntotcalc_fit)
    print('kI_fit:    ',kI_fit,'     kIcalc_fit:     ',kIcalc_fit)
    print('c_fit:     ',c_fit,'         ccalc_fit:      ',ccalc_fit)
    print('-------------------------------------------------------------------------------------------')
    print('mu,sigma,Ntot,kI,c')
    print('kI error: ',param_errors[3])
    print('kI_calc error: ',param_errors_calc[3])
    print('-------------------------------------------------------------------------------------------')
    
    conv = ConvFit(xdata_fit,xgauss, mu_fit, sigma_fit,A,kx,kxM,Ntot_fit,kI_fit,c_fit)
    conv_calc = ConvFit(xdata_fit,xgauss, mucalc_fit, sigmacalc_fit,A,kx,kxM,Ntotcalc_fit,kIcalc_fit,ccalc_fit)

    diff = int((len(xdata_fit) - len(conv)))
    lim=int(diff/2)
    print(diff)

    if ist_gerade(diff):
        xconv = xdata_fit[lim:][:-lim]
    else:
        xconv = xdata_fit[lim+1:][:-lim]

    # Create subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(7, 3))

    # Full range plot on the left
    ax1.step(xdata, ydata, color='k', label='LLAMA data')
    ax1.step(xconv, conv, color=purple, label='Fit $k_I$ with new model')
    ax1.step(xconv, conv_calc, color=cyan, label='Fit $k_{I,bounded}$ with new model')

    # Zoomed-in plot on the right
    ax2.step(xdata, ydata, color='k', label='LLAMA data')
    ax2.step(xconv, conv, color=purple, label='Fit $k_I$ with new model')
    ax2.step(xconv, conv_calc, color=cyan, label='Fit $k_{I,bounded}$ with new model')
    ax2.set_xlim(-500, 1000)
    legend1 = ax1.legend(fontsize = 9)
    legend1.set_title(Conname[i], prop = FontProperties(size=10))

    if i == 0:
        ax1.set_ylim(0.5,3e4)
        ax2.set_ylim(0.5,3e4)
    if i == 1:
        ax1.set_ylim(5,6e4)
        ax2.set_ylim(5,6e4)
    if i == 2:
        ax1.set_ylim(10,5e5)
        ax2.set_ylim(10,5e5)
    if i == 3:
        ax1.set_ylim(3,2e5)
        ax2.set_ylim(3,2e5)
        
        
    TwoPlots(time_label,"counts/40ns","no","log",1,"no","no",time_label,"counts/40ns","no","log",1,"no","no","Leibelt_kIFit_"+ path[i])

    kIarr.append(kI_fit)
    kIcalcarr.append(kIcalc_fit)
    kxtheoarr.append(kx)

kIarr = np.array(kIarr)
kIcalcarr = np.array(kIcalcarr)
kxtheoarr = np.array(kxtheoarr)


In [ ]:
#Fit to the data with fit parameters: kI - sigma,Ntot,c - with weighted bins and rebinned + Fit ohne Gauss
Distance = 15.

for i in range(0,len(path)):
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    print('i:',i,',',path[i])
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')

    #set parameters
    if i == 0:
        kx, kxM, A = 0, 0, 0
    else:
        kx = 2.9 * 1e-4 * xecon_mol[i]
        kxM = 2.9 * 1e-4 * xecon_mol[i]
        lambda1fit = 12.69 * 0.1/xecon_mol[i]
        lambda2fit = 741.4 * 0.1/xecon_mol[i]
        T = constants["Aconst"]*np.exp(-Distance/lambda1fit)+(1-constants["Aconst"])*np.exp(-Distance/lambda2fit)
        A = 1-T

    # Open the ROOT file and extract the histogram
    file = uproot.open("C:/Users/maria/Documents/Uni/Bachelorarbeit/Coden/out_stable/stable_" + path[i]+"/out.root")
    hist = file["hf2_4_1"]

    #  Get the bin contents and edges
    counts, bin_edges = hist.to_numpy()

    # Rebin
    rebin_factor = 2
    new_counts = np.add.reduceat(counts, np.arange(0, len(counts), rebin_factor))
    new_bin_edges = bin_edges[::rebin_factor]
    bin_centers = (new_bin_edges[:-1] + new_bin_edges[1:]) / 2

    # Generate bin centers with the specified step size
    step_size = new_bin_edges[1]-new_bin_edges[0]
    #print('stepsize: ',step_size)

    # Gauss x values
    xgauss = np.arange(-100, 110, step_size)
    #print('xgauss: ',xgauss)

    #x-values for the Fit and the data
    x_min, x_max = -200, 1e4
    x_pad_start, x_conv_max = -200,1e4

    #mu, sigma, Ntot, kI, c
    bounds = (
        [-100, 10, -np.inf, 1e-7, 0],  # Lower bounds
        [100, 100, np.inf, 1e-2, np.inf]  # Upper bounds
    )
    # boundscalc = (
    #     [-100, 10, -np.inf, constants["kI_lower_bound"], 0],  # Lower bounds
    #     [100, 100, np.inf, constants["kI_upper_bound"], np.inf]  # Upper bounds
    # )
    #Ntot,kI, c
    bounds_ohnegauss = (
        [-np.inf, 0., 0],
        [np.inf, 1e-2, np.inf]
    )
    initial_guesses_ohnegauss = [4e6, 3e-4, 20]

    if i == 0:
        xfit_min, xfit_max = -250, 10000
        initial_guesses = [1, 50, 4e6, 3e-4, 15]
    elif i == 1:
        xfit_min, xfit_max = -250, 10000
        initial_guesses = [1, 50, 4e6, 3e-4, 10]
    elif i == 2:
        xfit_min, xfit_max = -250, 5200
        initial_guesses = [1, 50, 4e6, 3e-4, 25]
    elif i == 3:
        xfit_min, xfit_max = -250, 2100
        initial_guesses = [1, 50, 4e6, 3e-4, 25]
        bounds = (
            [-100, 30, -np.inf, 1e-7, 0],  # Lower bounds
            [100, 50, np.inf, 1e-2, np.inf]  # Upper bounds
        )
        # boundscalc = (
        #     [-100, 30, -np.inf, constants["kI_lower_bound"], 0],  # Lower bounds
        #     [100, 50, np.inf, constants["kI_upper_bound"], np.inf]  # Upper bounds
        # )
    elif i == 4:
        xfit_min, xfit_max = -250, 2250
        initial_guesses = [1, 50, 4e6, 3e-4, 20]
        bounds = (
            [-np.inf, 30, -np.inf, 1e-7, 0],  # Lower bounds
            [np.inf, 50, np.inf, 1e-2, np.inf]  # Upper bounds
        )
        # boundscalc = (
        #     [-np.inf, 30, -np.inf, constants["kI_lower_bound"], 0],  # Lower bounds
        #     [np.inf, 50, np.inf, constants["kI_upper_bound"], np.inf]  # Upper bounds
        # )
    elif i == 5:
        xfit_min, xfit_max = -250, 1500
        initial_guesses = [1, 50, 4e6, 3e-4, 70]
        bounds = (
            [-100, -np.inf, -np.inf, 1e-7, 0],  # Lower bounds
            [100, np.inf, np.inf, 1e-2, np.inf]  # Upper bounds
        )
        # boundscalc = (
        #     [-100, -np.inf, -np.inf,constants["kI_lower_bound"], 0],  # Lower bounds
        #     [100, np.inf, np.inf, constants["kI_upper_bound"], np.inf]  # Upper bounds
        # )

    # Apply the mask to filter data for x_min to x_max
    mask = (bin_centers >= x_min) & (bin_centers <= x_max)
    xdata = bin_centers[mask]
    ydata = new_counts[mask]

    mask_conv = (bin_centers >= x_pad_start) & (bin_centers <= x_conv_max)
    xdata_conv = bin_centers[mask_conv]
    ydata_conv = new_counts[mask_conv]

    mask_fit = (bin_centers >= xfit_min) & (bin_centers <= xfit_max)
    xdata_fit = bin_centers[mask_fit]
    ydata_fit = new_counts[mask_fit]
      
    ydata_fit_chopped = ydata_fit[3:][:-2]

    # Weight: sqrt(#binentries)
    weight = np.where(ydata_fit_chopped == 0, np.inf, np.sqrt(ydata_fit_chopped))  
    weight_ohnegauss = np.where(ydata_fit == 0, np.inf, np.sqrt(ydata_fit)) 

    # Curve fitting
    popt_weighted, pcov_weighted = curve_fit(
        lambda t_, mu_fit, sigma_fit, Ntot_fit, kI_fit, c_fit: ConvFit(
            t_, xgauss, mu_fit, sigma_fit, A, kx, kxM, Ntot_fit, kI_fit, c_fit),
        xdata_fit, ydata_fit_chopped, p0=initial_guesses, bounds=bounds, sigma=weight, absolute_sigma=True
    )
    # poptcalc_weighted, pcovcalc_weighted = curve_fit(
    #     lambda t_, mucalc_fit, sigmacalc_fit, Ntotcalc_fit, kIcalc_fit, ccalc_fit: ConvFit(
    #         t_, xgauss, mucalc_fit, sigmacalc_fit, A, kx, kxM, Ntotcalc_fit, kIcalc_fit, ccalc_fit),
    #     xdata_fit, ydata_fit_chopped, p0=initial_guesses, bounds=boundscalc, sigma=weight, absolute_sigma=True
    # )
    poptohnegauss_weighted, pcovohnegauss_weighted = curve_fit(
        lambda t_, Ntotohnegauss_fit, kIohnegauss_fit, cohnegauss_fit: fitohnegauss(
            t_, A, kx, kxM, Ntotohnegauss_fit, kIohnegauss_fit, cohnegauss_fit),
        xdata_fit, ydata_fit, p0=initial_guesses_ohnegauss, bounds=bounds_ohnegauss, sigma=weight_ohnegauss, absolute_sigma=True
    )

    mu_fit, sigma_fit, Ntot_fit, kI_fit, c_fit = popt_weighted
    # mucalc_fit, sigmacalc_fit, Ntotcalc_fit, kIcalc_fit, ccalc_fit = poptcalc_weighted
    Ntotohnegauss_fit, kIohnegauss_fit, cohnegauss_fit = poptohnegauss_weighted

    param_errors = np.sqrt(np.diag(pcov_weighted))
    # param_errors_calc = np.sqrt(np.diag(pcovcalc_weighted))
    param_errors_ohnegauss = np.sqrt(np.diag(pcovohnegauss_weighted))
    
    # #print('-------------------------------------------------------------------------------------------')
    # print('Guesses: ','mu: ',initial_guesses[0],'sigma: ',initial_guesses[1],'k_I: ',initial_guesses[3],'c: ',initial_guesses[4])
    # print('mu_fit:    ',mu_fit,'        mucalc_fit:    ',mucalc_fit)
    # print('sigma_fit: ',sigma_fit,'          sigmacalc_fit:  ',sigmacalc_fit)
    # print('Ntot_fit:  ',Ntot_fit,'        Ntotcalc_fit:   ',Ntotcalc_fit)
    # print('kI_fit:    ',kI_fit,'     kIcalc_fit:     ',kIcalc_fit)
    # print('c_fit:     ',c_fit,'         ccalc_fit:      ',ccalc_fit)
    
    conv = ConvFit(xdata_fit,xgauss, mu_fit, sigma_fit,A,kx,kxM,Ntot_fit,kI_fit,c_fit)
    # conv_calc = ConvFit(xdata_fit,xgauss, mucalc_fit, sigmacalc_fit,A,kx,kxM,Ntotcalc_fit,kIcalc_fit,ccalc_fit)
    ohnegauss = fitohnegauss(xdata_fit,A,kx,kxM,Ntotohnegauss_fit,kIohnegauss_fit,cohnegauss_fit)

    diff = int((len(xdata_fit) - len(conv)))
    lim=int(diff/2)
    print(diff)
    
    if ist_gerade(diff):
        xconv = xdata_fit[lim:][:-lim]
    else:
        xconv = xdata_fit[lim+1:][:-lim]

    # Create subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(7, 3))

    # Full range plot on the left
    ax1.step(xdata, ydata, color='k', label='LLAMA data')
    ax1.step(xconv, conv, color=purple, label='Fit $k_I$ with new model')
    # ax1.step(xconv, conv_calc, color=cyan, label='Fit $k_{I,calc}$ with new model')
    ax1.step(xdata_fit, ohnegauss, color=green, label='Fit $k_I$ without convolution')

    # Zoomed-in plot on the right
    ax2.step(xdata, ydata, color='k', label='LLAMA data')
    ax2.step(xconv, conv, color=purple, label='Fit $k_I$ with new model')
    # ax2.step(xconv, conv_calc, color=cyan, label='Fit $k_{I,calc}$ with new model')
    ax2.step(xdata_fit, ohnegauss, color=green, label='Fit $k_I$ without convolution')
    ax2.set_xlim(-500, 1000)
    legend1 = ax1.legend(fontsize = 9)
    legend1.set_title(Conname[i], prop = FontProperties(size=10))

    if i == 0:
        ax1.set_ylim(0.5,1e4)
        ax2.set_ylim(0.5,1e4)
    if i == 1:
        ax1.set_ylim(3,3e4)
        ax2.set_ylim(3,3e4)
    if i == 2:
        ax1.set_ylim(10,7e4)
        ax2.set_ylim(10,7e4)
        
    TwoPlots(time_label,"counts/40ns","no","log",1,"no","no",time_label,"counts/40ns","no","log",1,"no","no","Leibelt_kIFit_ohnegauss_"+ path[i])


In [ ]:
#Fit to the data with fit parameters: kx - sigma,Ntot,c - with weighted bins and rebinned
#kI set to 4.28e-4
Distance = 15.
kIfit = 4.28e-4

kxarr = []
for i in range(0,len(path)): 
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    print('i:',i,',',path[i])
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    if i == 0:
        A = 0
    else:
        lambda1fit = 12.69 * 0.1/xecon_mol[i]
        lambda2fit = 741.4 * 0.1/xecon_mol[i]
        T = constants["Aconst"]*np.exp(-Distance/lambda1fit)+(1-constants["Aconst"])*np.exp(-Distance/lambda2fit)
        A = 1-T

    # Open the ROOT file and extract the histogram
    file = uproot.open("C:/Users/maria/Documents/Uni/Bachelorarbeit/Coden/out_stable/stable_" + path[i]+"/out.root")
    hist = file["hf2_4_1"]

    #  Get the bin contents and edges
    counts, bin_edges = hist.to_numpy()

    # Rebin
    rebin_factor = 2
    new_counts = np.add.reduceat(counts, np.arange(0, len(counts), rebin_factor))
    new_bin_edges = bin_edges[::rebin_factor]
    bin_centers = (new_bin_edges[:-1] + new_bin_edges[1:]) / 2

    # Generate bin centers with the specified step size
    step_size = new_bin_edges[1]-new_bin_edges[0]
    #print('stepsize: ',step_size)

    # Gauss x values
    xgauss = np.arange(-100, 110, step_size)
    #print('xgauss: ',xgauss)

    #x-values for the Fit and the data
    x_min, x_max = -250, 1e4
    x_pad_start, x_conv_max = -250,1e4

    #mu, sigma, kx, Ntot, c
    bounds = (
        [-100,10,0,0,0],  # Lower bounds
        [100,100,1,np.inf,np.inf]  # Upper bounds
    )
    if i == 0:
        xfit_min, xfit_max = -250, 10000
        initial_guesses = [1, 50, 2.9 * 1e-4 * xecon_mol[0],4e6, 2]
    elif i == 1:
        xfit_min, xfit_max = -250, 10000
        initial_guesses = [1, 50, 2.9 * 1e-4 * xecon_mol[1],4e6, 10]
        bounds = (
            [-100, 10, 0 ,0, 0],  # Lower bounds
            [100, 100, 1e-2 ,np.inf, np.inf]  # Upper bounds
        )
    elif i == 2:
        xfit_min, xfit_max = -250, 5200
        initial_guesses = [1, 50, 2.9 * 1e-4 * xecon_mol[2],4e6, 25]
    elif i == 3:
        xfit_min, xfit_max = -250, 2100
        initial_guesses = [1, 50, 2.9 * 1e-4 * xecon_mol[3],4e6, 25]
        bounds = (
            [-100, 30, 0 ,-np.inf, 0],  # Lower bounds
            [100, 50, 1 ,np.inf, np.inf]  # Upper bounds
        )
    elif i == 4:
        xfit_min, xfit_max = -250, 2250
        initial_guesses = [1, 50, 2.9 * 1e-4 * xecon_mol[4],4e6, 20]
        bounds = (
            [-100,30,0,-np.inf,0],  # Lower bounds
            [100,50,1,np.inf,np.inf]  # Upper bounds
        )
    elif i == 5:
        xfit_min, xfit_max = -250, 1500
        initial_guesses = [1, 50, 2.9 * 1e-4 * xecon_mol[5],4e6, 70]
        bounds = (
            [-100,30,0,-np.inf,0],  # Lower bounds
            [100,50,1,np.inf,np.inf]  # Upper bounds
        )
    # Apply the mask to filter data for x_min to x_max
    mask = (bin_centers >= x_min) & (bin_centers <= x_max)
    xdata = bin_centers[mask]
    ydata = new_counts[mask]

    mask_conv = (bin_centers >= x_pad_start) & (bin_centers <= x_conv_max)
    xdata_conv = bin_centers[mask_conv]
    ydata_conv = new_counts[mask_conv]

    mask_fit = (bin_centers >= xfit_min) & (bin_centers <= xfit_max)
    xdata_fit = bin_centers[mask_fit]
    ydata_fit = new_counts[mask_fit]
      
    ydata_fit_chopped = ydata_fit[3:][:-2]

    # Weight: sqrt(#binentries)
    weight = np.where(ydata_fit_chopped == 0, np.inf, np.sqrt(ydata_fit_chopped))

    # Curve fitting
    popt_weighted, pcov_weighted = curve_fit(
        lambda  t_,mu_fit,sigma_fit,kx_fit,Ntot_fit,c_fit: ConvFit(t_, xgauss, mu_fit, sigma_fit,A,kx_fit,kx_fit,Ntot_fit,kIfit,c_fit), 
        xdata_fit, 
        ydata_fit_chopped, 
        p0=initial_guesses,
        bounds=bounds, 
        sigma=weight, 
        absolute_sigma=True
    )
    mu_fit,sigma_fit,kx_fit,Ntot_fit,c_fit = popt_weighted
    param_errors = np.sqrt(np.diag(pcov_weighted))
    #print('-------------------------------------------------------------------------------------------')
    print('Guesses: ','mu: ',initial_guesses[0],'sigma: ',initial_guesses[1],'k_x: ',initial_guesses[2],'c: ',initial_guesses[4])
    print('mu_fit:    ',mu_fit)
    print('sigma_fit: ',sigma_fit)
    print('kx_fit: ',kx_fit)
    print('Ntot_fit:  ',Ntot_fit)
    print('c_fit:     ',c_fit)

    print('kx error: ',param_errors[2])
    
    conv = ConvFit(xdata_fit,xgauss, mu_fit, sigma_fit,A,kx_fit,kx_fit,Ntot_fit,kIfit,c_fit)

    diff = int((len(xdata_fit) - len(conv)))
    lim=int(diff/2)

    if ist_gerade(diff):
        xconv = xdata_fit[lim:][:-lim]
    else:
        xconv = xdata_fit[lim+1:][:-lim]

    # Create subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(7,3))
    if i == 1:
        ax1.set_ylim(1,2e4)
        ax2.set_ylim(1,2e4)
    if i == 2:
        ax1.set_ylim(10,2e4)
        ax2.set_ylim(10,2e4)
    if i == 3:
        ax1.set_ylim(5,5e4)
        ax2.set_ylim(5,5e4)
    # Full range plot on the left
    ax1.step(xdata, ydata, color='k', label='LLAMA data')
    ax1.step(xconv, conv, color=cyan, label='Fit of $k_x$ with new model')

    # Zoomed-in plot on the right
    ax2.step(xdata, ydata, color='k', label='LLAMA data')
    ax2.step(xconv, conv, color=cyan, label='conv')
    ax2.set_xlim(-500, 1000)
    legend1 = ax1.legend(fontsize = 9)
    legend1.set_title(Conname[i], prop = FontProperties(size=10))

    TwoPlots(time_label,countlabel,"no","log",1,"no","no",time_label,countlabel,"no","log",1,"no","no","Leibelt_kxFit_"+ path[i])

    kxarr.append(kx_fit)
kxarr = np.array(kxarr)

In [ ]:
#kx and kI from fit vs. xecon concentration
print('xecon_mol: ',xecon_mol)
print('kIarr:     ',kIarr)
print('kIcalcarr: ',kIcalcarr)
print('kxarr:     ',kxarr)
print('kxtheoarr: ',kxtheoarr)

xaxis = xecon_mol
y1 = kIarr
y2 = kIcalcarr
ybound1 = np.ones(6, dtype=int)*constants["kI_lower_bound"]
ybound2 = np.ones(6, dtype=int)*constants["kI_upper_bound"]
y3 = kxarr*3.3
y4 = kxtheoarr*3.3

fig, ax1 = plt.subplots(figsize=(4,3))
ax1.plot(xaxis, ybound1,"--", color = 'k', label=r'$k_I$ bounds')
ax1.plot(xaxis, ybound2,"--", color = 'k')
ax1.plot(xaxis, y1, ".",color = darkblue, label=r'$k_I$ from fit')
ax1.plot(xaxis, y2, ".", color = green, label=r'$k_{I,bounded}$ from fit')
ax1.legend(fontsize = 9)
formatter = ticker.ScalarFormatter()
formatter.set_powerlimits((-3, 4))  # Werte außerhalb dieses Bereichs werden in wissenschaftlicher Notation angezeigt
ax1.yaxis.set_major_formatter(formatter)
ax1.set_ylim(-5e-5,7e-4)
OnePlot(Xe_mol_label,"$k_I$ [1/ns]","no","no",2,Xe_g_label,"no","Leibelt_kIvsxecon")

fig, ax1 = plt.subplots(figsize=(4,3))
ax1.plot(xaxis, y4, 'k-', label='linear approach')
ax1.plot(xaxis, y3, ".", color = purple, label='$k_x$ from fit')
ax1.legend(fontsize = 9)
OnePlot(Xe_mol_label,r"$k_x$ [1/(ns $\mu$mol/mol)]","no","no",2,Xe_g_label,"no","Leibelt_kxvsxecon")

In [ ]:
#pey mit kx von fit compared to data
Distance = 0.93
Ntot = 1

kIfit_average = (kIarr[0]+kIarr[1]+kIarr[2])/3
print("kI_average of reasonable fit values",kIfit_average)
kxfit = kxarr
kx = 2.9 * 1e-4 * xeconpey_mol

yAllarr_fit  = []
for i in range(0,len(kxarr)):
    for j in range(0,len(Type)-1):
        if Type[j] == 'S':
            yAllS_fit = Lxekx(xecon_mol[i],Type[j],Distance,Ntot,kIfit_average,kxfit[i])*constants["PDE128"]
            yAllS_fit_norm = Lxekx(300/3.3,Type[j],Distance,Ntot,kIfit_average,kxfit[5])*constants["PDE128"]
        elif Type[j] == 'T':
            yAllT_fit = Lxekx(xecon_mol[i],Type[j],Distance,Ntot,kIfit_average,kxfit[i])*constants["PDE128"]
            yAllT_fit_norm = Lxekx(300/3.3,Type[j],Distance,Ntot,kIfit_average,kxfit[5])*constants["PDE128"]
        elif Type[j] == 'M':
            yAllM_fit = Lxekx(xecon_mol[i],Type[j],Distance,Ntot,kIfit_average,kxfit[i])*constants["PDE150"]
            yAllM_fit_norm = Lxekx(300/3.3,Type[j],Distance,Ntot,kIfit_average,kxfit[5])*constants["PDE150"]
        elif Type[j] == 'X':
            yAllX_fit = Lxekx(xecon_mol[i],Type[j],Distance,Ntot,kIfit_average,kxfit[i])*constants["PDE175"]
            yAllX_fit_norm = Lxekx(300/3.3,Type[j],Distance,Ntot,kIfit_average,kxfit[5])*constants["PDE175"]
        else:
            PDE = -2000
    yAll_fit = yAllS_fit + yAllT_fit + yAllM_fit+ yAllX_fit
    yAllarr_fit.append(yAll_fit)
yAll_fit_norm = yAllS_fit_norm + yAllT_fit_norm + yAllM_fit_norm + yAllX_fit_norm
yAll_fit_norm_val = 1 / yAll_fit_norm

for j in range(0,len(Type)-1):
    if Type[j] == 'S':
        yAllS = Lxe(xeconpey_mol,Type[j],Distance,Ntot,constants["kI_average"],kx)*constants["PDE128"]
        yAllS_norm = Lxekx(300/3.3,Type[j],Distance,Ntot,constants["kI_average"],2.9 * 1e-4 * (300/3.3))*constants["PDE128"]
    elif Type[j] == 'T':
        yAllT = Lxe(xeconpey_mol,Type[j],Distance,Ntot,constants["kI_average"],kx)*constants["PDE128"]
        yAllT_norm = Lxekx(300/3.3,Type[j],Distance,Ntot,constants["kI_average"],2.9 * 1e-4 * (300/3.3))*constants["PDE128"]
    elif Type[j] == 'M':
        yAllM = Lxe(xeconpey_mol,Type[j],Distance,Ntot,constants["kI_average"],kx)*constants["PDE150"]
        yAllM_norm = Lxekx(300/3.3,Type[j],Distance,Ntot,constants["kI_average"],2.9 * 1e-4 * (300/3.3))*constants["PDE150"]
    elif Type[j] == 'X':
        yAllX = Lxe(xeconpey_mol,Type[j],Distance,Ntot,constants["kI_average"],kx)*constants["PDE175"]
        yAllX_norm = Lxekx(300/3.3,Type[j],Distance,Ntot,constants["kI_average"],2.9 * 1e-4 * (300/3.3))*constants["PDE175"]
    else:
        PDE = -2000

yAll = yAllS + yAllT + yAllM + yAllX

yAll_norm = yAllS_norm+yAllT_norm+yAllM_norm+yAllX_norm
norm_val = 1/yAll_norm

yAllarr_fit = np.array(yAllarr_fit)

fig, (ax1,ax2) = plt.subplots(1, 2, figsize=(7, 3))
ax1.plot(xeconpey_mol, yAll*norm_val, 'k', label = 'New model')
ax1.errorbar(xecon_mol, Data_norm, Data_norm_error,fmt = '.',color = Color[6], label = 'LLAMA data')
ax1.plot(xecon_mol, yAllarr_fit*yAll_fit_norm_val, '.', color = darkblue, label = 'New model with $k_{x,fit}$')

ax2.plot(xeconpey_mol, yAll*norm_val, "k", label = 'New model')
ax2.errorbar(xecon_mol_log, Data_norm, Data_norm_error,fmt = '.',color = Color[6], label = 'LLAMA data')
ax2.plot(xecon_mol_log, yAllarr_fit*yAll_fit_norm_val, '.', color = darkblue, label = 'New model with $k_{x,fit}')
legend1 = ax1.legend(fontsize=9)
TwoPlots(Xe_mol_label,pe_label,"no","no",2,Xe_g_label,"no",Xe_mol_label,pe_label,"log","no",2,Xe_g_label,"log","Leibelt_pey_Data_kxkIfit")

In [ ]:
#Detector Distance Dependency of the Photon Arrival Time Spectra
print('--------------------------------------------------------')
print('Photon arrival time spectra - t: 0-10000 - Distance')
print('--------------------------------------------------------')
xecon_mario = np.array([3,50])/3.3
mario_name = np.array([r"0.91 $\mu$mol/mol",r"15.15 $\mu$mol/mol"])
tmax = 10000
t = np.array(range(0, tmax))
kIfit = 4.28e-4
kx = [kxarr[1],kxarr[3]]
kxM = kx
Ntot = 1

yAll0 = []
yAll1 = []
for j in range(1,len(x)):
    strDistance = str(x[j])
    Distance = x[j]
    print(Distance)

    lambda1fit0 = 12.69 * 0.1/xecon_mario[0]
    lambda2fit0 = 741.4 * 0.1/xecon_mario[0]
    T0 = constants["Aconst"]*np.exp(-Distance/lambda1fit0)+(1-constants["Aconst"])*np.exp(-Distance/lambda2fit0)
    A0 = 1-T0
    lambda1fit1 = 12.69 * 0.1/xecon_mario[1]
    lambda2fit1 = 741.4 * 0.1/xecon_mario[1]
    T1 = constants["Aconst"]*np.exp(-Distance/lambda1fit1)+(1-constants["Aconst"])*np.exp(-Distance/lambda2fit1)
    A1 = 1-T1

    y0 = l(t,A0,kx[0],kxM[0],'All',Ntot,kIfit)
    y1 = l(t,A1,kx[1],kxM[1],'All',Ntot,kIfit)
    yAll0.append(y0)
    yAll1.append(y1)

yAll0 = np.array(yAll0)
yAll1 = np.array(yAll1)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(7,3))
ax1.plot(t, yAll0[0], 'k', label = str(x[1])+" cm")
ax2.plot(t, yAll1[0], 'k', label = str(x[1])+" cm")
ax1.plot(t, yAll0[1], purple, label = str(x[2])+" cm")
ax2.plot(t, yAll1[1], purple, label = str(x[2])+" cm")
ax1.plot(t, yAll0[2], yellow, label = str(x[3])+" cm")
ax2.plot(t, yAll1[2], yellow, label = str(x[3])+" cm")
ax1.set_xlim(-500,4e3)
ax2.set_xlim(-500,4e3)
ax1.set_ylim(1e-6,1e-2)
ax2.set_ylim(1e-6,1e-2)

legend1 = ax1.legend(fontsize=9)
legend1.set_title(mario_name[0], prop = FontProperties(size=10))
legend2 = ax2.legend(fontsize=9)
legend2.set_title(mario_name[1], prop = FontProperties(size=10))
TwoPlots(time_label,countlabel,"no","log",1,"no","no",time_label,countlabel,"no","log",1,"no","no","Leibelt_Photonarr_distance")


In [ ]:
#Detector Distance Dependency of the Photoelectron Yield
Ntot = 1
kx = kxarr
yAllarr = []
kIfit = 4.28e-4

for i in range(0,len(x)):
    Distance = x[i]
    strDistance = str(Distance)    
    for j in range(0,len(Type)-1):
        if Type[j] == 'S':
            yAllS = Lxe(xecon_mol,Type[j],Distance,Ntot,kIfit,kx)*constants["PDE128"]
        elif Type[j] == 'T':
            yAllT = Lxe(xecon_mol,Type[j],Distance,Ntot,kIfit,kx)*constants["PDE128"]
        elif Type[j] == 'M':
            yAllM = Lxe(xecon_mol,Type[j],Distance,Ntot,kIfit,kx)*constants["PDE150"]
        elif Type[j] == 'X':
            yAllX = Lxe(xecon_mol,Type[j],Distance,Ntot,kIfit,kx)*constants["PDE175"]
        else:
            PDE = -2000

    yAll = yAllS + yAllT + yAllM + yAllX
    yAllarr.append(yAll)
yAllarr = np.array(yAllarr)

norm_val = 1/yAllarr[0][5]

fig, (ax1,ax2) = plt.subplots(1, 2, figsize=(7, 3))
colordistance = [darkblue,"k",purple,yellow]
for j in range(0,len(yAllarr)):
    Distance = x[j]
    strDistance = str(Distance)
    
    print(strDistance)

    ax1.plot(xecon_mol, yAllarr[j]*norm_val, ".",color=colordistance[j], label = strDistance + " cm")
    ax2.plot(xecon_mol_log, yAllarr[j]*norm_val, ".",color=colordistance[j], label = strDistance + " cm")
    legend1 = ax1.legend(fontsize=9)
TwoPlots(Xe_mol_label,pe_label,"no","no",2,Xe_g_label,"no",Xe_mol_label,pe_label,"log","no",2,Xe_g_label,"log","Leibelt_pey_withoutData_distance")


In [ ]:
#show that the impurities (kI) cause the dip in the photoelectron yield
kI_values = np.linspace(0, 1e-3, 1000)  # 5 values from 0 to 0.01 for demonstration

xecon_mol = np.array([0,3,10,50,100,300]) / 3.3
Distance = 0.93
Ntot = 1
kx = kxarr
yAllarr = np.zeros((len(kI_values), len(xecon_mol)))  # 2D array to store yAll values

for i in range(0,len(xecon_mol)):
    for idx, kI in enumerate(kI_values):
        for j in range(0,len(Type) - 1):
            if Type[j] == 'S':
                yAllS = Lxekx(xecon_mol[i], Type[j], Distance, Ntot, kI, kx[i]) * constants["PDE128"]
            elif Type[j] == 'T':
                yAllT = Lxekx(xecon_mol[i], Type[j], Distance, Ntot, kI, kx[i]) * constants["PDE128"]
            elif Type[j] == 'M':
                yAllM = Lxekx(xecon_mol[i], Type[j], Distance, Ntot, kI, kx[i]) * constants["PDE150"]
            elif Type[j] == 'X':
                yAllX = Lxekx(xecon_mol[i], Type[j], Distance, Ntot, kI, kx[i]) * constants["PDE175"]
            else:
                PDE = -2000
        
        yAll = yAllS + yAllT + yAllM + yAllX
        
        yAllarr[idx, i] = yAll

# Normalize each function by its respective value at xecon_mol[5]
norm_vals = yAllarr[:, 5]
yAllarr_normalized = yAllarr / norm_vals[:, np.newaxis]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(7, 3))

colormap = plt.get_cmap("viridis", len(kI_values))

for idx, kI in enumerate(kI_values):
    color = colormap(idx)
    label = f"kI={kI:.5f}"
    ax1.plot(xecon_mol, yAllarr_normalized[idx, :], ".", color=color, label=label)
    ax2.plot(xecon_mol_log, yAllarr_normalized[idx, :], ".", color=color, label=label)
#legend1 = ax1.legend(fontsize=9, ncol=2)
TwoPlots(Xe_mol_label, pe_label, "no", "no", 2, Xe_g_label, "no", Xe_mol_label, pe_label, "log", "no", 2, Xe_g_label, "log", "Leibelt_pey_dipkI")